In [1]:
import sys

from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize, RandomHorizontalFlip, RandomRotation, Resize
import torch

sys.path.insert(0, '..')

from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
model = SiameseDuplicateImageNetwork().to(device)

train_dataset = DuplicateImageDataset('../data/Airbnb Data/Training Data', 
                                      transforms=[
                                          Resize((256, 256), antialias=True),
                                          Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ],
                                      upsample_transforms_dict={
                                          'hflip': RandomHorizontalFlip(p=1),
                                          'anticlockwise_rot': RandomRotation((5, 5)),
                                          'clockwise_rot': RandomRotation((-5, -5))
                                      })
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = DuplicateImageDataset('../data/Airbnb Data/Test Data', 
                                      transforms=[
                                          Resize((256, 256), antialias=True),
                                          Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ],
                                      upsample_transforms_dict={
                                          'hflip': RandomHorizontalFlip(p=1),
                                          'anticlockwise_rot': RandomRotation((5, 5)),
                                          'clockwise_rot': RandomRotation((-5, -5))
                                      })
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

print(model)

SiameseDuplicateImageNetwork(
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): Sequential(
        (0): Sequential(
          (0): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): Hardswish()
            )
            (1): InvertedResidual(
              (block): Sequential(
                (0): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                  (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                  (2): ReLU(inplace=True)
                )
                (1): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (1): BatchNorm2d(16, eps=0.001, mom

In [4]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(
    [params for params in model.parameters() if params.requires_grad],
    lr=1e-4,
    alpha=0.9
)

In [5]:
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train_loop(train_dataloader, loss_fn, optimizer, device)
    model.test_loop(test_dataloader, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
loss: 0.735494  [   64/199552]
loss: 0.615822  [ 6464/199552]
loss: 0.509134  [12864/199552]
loss: 0.571220  [19264/199552]
loss: 0.596071  [25664/199552]
loss: 0.551094  [32064/199552]
loss: 0.522529  [38464/199552]
loss: 0.558152  [44864/199552]
loss: 0.518892  [51264/199552]
loss: 0.531083  [57664/199552]
loss: 0.531312  [64064/199552]
loss: 0.501772  [70464/199552]
loss: 0.584267  [76864/199552]
loss: 0.479867  [83264/199552]
loss: 0.574787  [89664/199552]
loss: 0.518671  [96064/199552]
loss: 0.556276  [102464/199552]
loss: 0.540317  [108864/199552]
loss: 0.516494  [115264/199552]
loss: 0.578828  [121664/199552]
loss: 0.524145  [128064/199552]
loss: 0.532527  [134464/199552]
loss: 0.511698  [140864/199552]
loss: 0.546298  [147264/199552]
loss: 0.530326  [153664/199552]
loss: 0.527326  [160064/199552]
loss: 0.535651  [166464/199552]
loss: 0.503331  [172864/199552]
loss: 0.495948  [179264/199552]
loss: 0.501798  [185664/199552]
loss: 0.497520  